In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms
import torchvision.models as models
from tqdm import tqdm  

In [ ]:
#Add the test dataloader with output with (eeg,mel,mel,mel,...)  eeg = [64,_,_] 64 channels, mel = [10,_,_] 10 channels 
# save the test dataloader with variable dataloader_test
# this code will only work if the batch_size = 0 do the shape of the eeg and mel should be an 3D array 







In [ ]:
class CustomResNet(nn.Module):
    def __init__(self, input_channels):
        super(CustomResNet, self).__init__()
        self.resnet = models.resnet34(pretrained=False)
        self.resnet.conv1 = nn.Conv2d(input_channels, 64, kernel_size=7, stride=2, padding=3, bias=False)

    def forward(self, x):
        x = self.resnet(x)
        x = x.view(x.size(0),-1)
        return x

In [ ]:
device = 'cuda'
resnet_eeg = CustomResNet(64)
resnet_mel = CustomResNet(10)
resnet_eeg = resnet_eeg.to(device)
resnet_mel = resnet_mel.to(device)

In [ ]:
resnet_eeg.load_state_dict(torch.load('resnet_eeg.pth'))
resnet_mel.load_state_dict(torch.load('resnet_mel.pth'))

In [ ]:
# criterion
def euclidean_dist(eeg, mel):
    distance = np.sqrt(np.dot(eeg - mel, (eeg - mel).T))
    return distance

In [ ]:
with torch.no_grad():
    for list in dataloader_test:
        eeg = list[0].to(device)
        output_eeg = resnet_eeg(eeg)
        # lowest_distance = np.inf
        # matched_id = 0
        distances = []
        
        for i in range(1,len(list)):
            output_mel = resnet_mel(list[i].to(device))
            distance = euclidean_dist(output_eeg,output_mel)
            distances.append(distance)

        print("matched Index" , distances.index(min(distances)))
            
        
            
        
                
        